In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import pandas as pd

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display


2023-11-27 11:43:40.344018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Model Initialization
model_name = "movenet_lightning"
# module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
model = module.signatures['serving_default']


In [16]:
# Choose image file and preprocess

image_path = "../clean_data/TRAIN/downdog/00000128.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


In [17]:
# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
xyz = outputs["output_0"].numpy().reshape(51).tolist()


In [18]:
# Helper method for drawing landmarks
def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot


# Extract landmarks from all files in a directory

In [241]:
# Get list of image file names in given directory
dir_path_train_dd = "../clean_data/VAL/warrior2/warrior2_right"
file_names = []
for entry in os.listdir(dir_path_train_dd):
    file_names.append(entry)


In [242]:
# Create DF of xyz values
df = []
xyz = []
for file in file_names:
    dir_path_train_dd = f"../clean_data/VAL/warrior2/warrior2_right/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
        print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        to_df = outputs["output_0"].numpy().reshape(51).tolist()
        to_df.insert(0, dir_path_train_dd)
        df.append(to_df)

        # Append the unflattened data for angle analysis
        # xyz.append(outputs["output_0"].numpy()[0][0])


../clean_data/VAL/warrior2/warrior2_right/00000010.jpg
../clean_data/VAL/warrior2/warrior2_right/00000005.jpg
../clean_data/VAL/warrior2/warrior2_right/00000011.jpg
../clean_data/VAL/warrior2/warrior2_right/00000006.png
../clean_data/VAL/warrior2/warrior2_right/00000016.jpg
../clean_data/VAL/warrior2/warrior2_right/00000015.jpg
../clean_data/VAL/warrior2/warrior2_right/00000000.jpg
../clean_data/VAL/warrior2/warrior2_right/00000040.jpg
../clean_data/VAL/warrior2/warrior2_right/00000018.jpg
../clean_data/VAL/warrior2/warrior2_right/00000032.jpg
../clean_data/VAL/warrior2/warrior2_right/00000033.jpg
../clean_data/VAL/warrior2/warrior2_right/00000034.jpg
../clean_data/VAL/warrior2/warrior2_right/00000020.jpg
../clean_data/VAL/warrior2/warrior2_right/00000035.jpg


In [145]:
model_name = "movenet_lightning"
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

def get_xyz(file_path, module):
    """
    This function takes a filepath for an image, and returns a list of xyz
    coordinates of the landmarks
    """
    input_size = 192

    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image)
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
    input_image = input_image[..., :3]
    input_image = tf.cast(input_image, dtype=tf.int32)

    model = module.signatures["serving_default"]
    outputs = model(input_image)

    xyz = outputs["output_0"].numpy()

    return xyz


In [243]:
# Change to DataFrame
a = pd.DataFrame(df)

# Add extra y1 and y2 labels
a["y_main"] =  "warrior2"
a["y_sub"] = "warrior2_right"


In [244]:
# Run the below just once when creating test_landmark_all for the first time
# val_landmark_all = a


In [245]:
val_landmark_all = pd.concat([val_landmark_all, a], ignore_index=True)
val_landmark_all.shape


(144, 54)

In [246]:
val_landmark_all.tail()


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,y_main,y_sub
139,../clean_data/VAL/warrior2/warrior2_right/0000...,0.318528,0.554444,0.733416,0.306447,0.540627,0.627944,0.304539,0.550032,0.782912,...,0.679014,0.485207,0.755016,0.242915,0.634383,0.753780,0.680964,0.787631,warrior2,warrior2_right
140,../clean_data/VAL/warrior2/warrior2_right/0000...,0.312817,0.576883,0.735261,0.295354,0.562783,0.694497,0.291925,0.563010,0.623199,...,0.719067,0.770251,0.835029,0.160283,0.785591,0.829874,0.747690,0.856841,warrior2,warrior2_right
141,../clean_data/VAL/warrior2/warrior2_right/0000...,0.295378,0.429856,0.718781,0.282970,0.429444,0.850737,0.282381,0.418199,0.748058,...,0.237722,0.730614,0.723085,0.530338,0.596994,0.703935,0.135520,0.728457,warrior2,warrior2_right
142,../clean_data/VAL/warrior2/warrior2_right/0000...,0.256058,0.591267,0.738903,0.239507,0.578456,0.635123,0.238964,0.574713,0.690047,...,0.283504,0.860723,0.797230,0.731032,0.826005,0.808969,0.099310,0.839227,warrior2,warrior2_right
143,../clean_data/VAL/warrior2/warrior2_right/0000...,0.293906,0.580950,0.300048,0.274271,0.572011,0.388846,0.275559,0.581232,0.488962,...,0.765567,0.620287,0.828237,0.245025,0.553020,0.857876,0.786115,0.771510,warrior2,warrior2_right


In [247]:
col_dict = {i: f"{chr(88 + (i-1) % 3)}{1 + (i-1) // 3}" for i in range(1, 52)}
col_dict
val_landmark_all.rename(columns=col_dict, inplace=True)
val_landmark_all.rename(columns={0: "file_path"}, inplace=True)

# val_landmark_all.to_csv("val_landmark_all_raw.csv", index=False)


In [248]:
test_df = pd.read_csv("test_landmark_all_raw.csv")
test_df


,file_path,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,...,Y15,Z15,X16,Y16,Z16,X17,Y17,Z17,y_main,y_sub
0,../clean_data/TEST/downdog/00000107.jpg,0.692118,0.609694,0.497758,0.688887,0.622062,0.458212,0.686494,0.621184,0.352763,...,0.378536,0.780648,0.691302,0.303898,0.578037,0.701163,0.301808,0.612833,downdog,downdog
1,../clean_data/TEST/downdog/00000067.jpg,0.661082,0.470409,0.579710,0.667057,0.447470,0.487697,0.667951,0.452278,0.580914,...,0.769717,0.572977,0.728744,0.913877,0.522456,0.704917,0.880844,0.675006,downdog,downdog
2,../clean_data/TEST/downdog/00000098.jpg,0.645959,0.391323,0.402054,0.649634,0.374465,0.531388,0.650784,0.375626,0.614023,...,0.769738,0.672423,0.704125,0.916061,0.816758,0.697093,0.909319,0.635933,downdog,downdog
3,../clean_data/TEST/downdog/00000099.jpg,0.661082,0.470409,0.579710,0.667057,0.447470,0.487697,0.667951,0.452278,0.580914,...,0.769717,0.572977,0.728744,0.913877,0.522456,0.704917,0.880844,0.675006,downdog,downdog
4,../clean_data/TEST/downdog/00000106.jpg,0.799238,0.571565,0.569596,0.797147,0.581129,0.629631,0.794709,0.584344,0.549455,...,0.276492,0.776100,0.849364,0.195126,0.650042,0.847213,0.176801,0.556480,downdog,downdog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,../clean_data/TEST/warrior2/warrior2-right/000...,0.217694,0.584351,0.751712,0.201037,0.580980,0.728456,0.198874,0.559794,0.457065,...,0.347593,0.903084,0.827565,0.775951,0.910766,0.793895,0.184491,0.877949,warrior2,warrior2-right
150,../clean_data/TEST/warrior2/warrior2-right/000...,0.198384,0.601135,0.815434,0.179533,0.595386,0.807498,0.179744,0.581226,0.780854,...,0.303325,0.872351,0.793069,0.800969,0.900845,0.796424,0.100702,0.868092,warrior2,warrior2-right
151,../clean_data/TEST/warrior2/warrior2-right/000...,0.217694,0.584351,0.751712,0.201037,0.580980,0.728456,0.198874,0.559794,0.457065,...,0.347593,0.903084,0.827565,0.775951,0.910766,0.793895,0.184491,0.877949,warrior2,warrior2-right
152,../clean_data/TEST/warrior2/warrior2-right/000...,0.397776,0.351683,0.432641,0.389028,0.355761,0.554081,0.387997,0.347120,0.608249,...,0.336019,0.157691,0.598191,0.428262,0.314101,0.633231,0.214439,0.417303,warrior2,warrior2-right


In [273]:
cols = test_df.columns.tolist()
new_cols = []
for col in cols:
    if "Z" in col:
        col = "C" + col[1:]
    new_cols.append(col)

changer = dict(zip(cols, new_cols))

test_df_new = test_df.rename(columns=changer)
train_df_new = (pd.read_csv("train_landmark_all_raw.csv")).rename(columns=changer)
val_df_new = (pd.read_csv("val_landmark_all_raw.csv")).rename(columns=changer)


In [274]:
test_df_new.to_csv("test_landmark_all_raw.csv", index=False)
train_df_new.to_csv("train_landmark_all_raw.csv", index=False)
val_df_new.to_csv("val_landmark_all_raw.csv", index=False)


In [279]:
xy_cols = []

for col in cols:
    if "Z" not in col:
        xy_cols.append(col)

test_df_xy = test_df_new[xy_cols]
train_df_xy = train_df_new[xy_cols]
val_df_xy = val_df_new[xy_cols]

test_df_xy.to_csv("test_landmark_xy_raw.csv", index=False)
train_df_xy.to_csv("train_landmark_xy_raw.csv", index=False)
val_df_xy.to_csv("val_landmark_xy_raw.csv", index=False)


In [9]:
"""Below used to rename some elements in the df, no longer needed"""
# cols = {
#     "warrior2-left": "warrior2_left",
#     "warrior2-right": "warrior2_right"
# }

# df = pd.read_csv("train_landmark_all_raw.csv")
# df_renamed = df.replace(cols)

# df_renamed.to_csv("train_landmark_all_raw.csv", index=False)


In [1]:
l = [[77, 33, 17],
 [89, 84, 55],
 [33, 80, 9],
 [9, 75, 84],
 [29, 42, 7],
 [89, 61, 22],
 [71, 2, 5],
 [71, 68, 6],
 [45, 84, 20],
 [35, 38, 63],
 [66, 28, 96],
 [85, 80, 88],
 [11, 45, 66],
 [78, 39, 15],
 [15, 13, 30],
 [87, 71, 20],
 [80, 18, 45]]


In [5]:
l2 = np.array(l).reshape(51).tolist()
len(l2)


51